This is for Chapter 9 [Winter 2020 of Statistical Rethinking](https://github.com/rmcelreath/stat_rethinking_2020/tree/main/homework) using Python. A public repo using `pmyc3` has been created [here](https://github.com/pymc-devs/resources/tree/master/Rethinking_2) that also uses Python. It will helpful for me to do these exercises myself, but that repo will serve as a helpful reference.

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pymc3 as pm
import scipy.stats as stats
import seaborn as sns

from scipy.optimize import curve_fit

In [2]:
%load_ext nb_black
%config InlineBackend.figure_format = 'retina'
%load_ext watermark
RANDOM_SEED = 8927
np.random.seed(RANDOM_SEED)
az.style.use("arviz-darkgrid")
az.rcParams["stats.hdi_prob"] = 0.89  # sets default credible interval used by arviz

<IPython.core.display.Javascript object>

In [3]:
sns.set_context("talk")
os.getcwd()

'/Users/blacar/Documents/ds_projects/stats_rethinking/repo'

<IPython.core.display.Javascript object>

In [4]:
def standardize(x):
    x = (x - np.mean(x)) / np.std(x)
    return x

<IPython.core.display.Javascript object>

# Easy problems

9E1. Which of the following is a requirement of the simple Metropolis algorithm?

1. The parameters must be discrete. (no)
2. The likelihood function must be Gaussian. (no - none of the MC algorithms have this assumption)
3. The proposal distribution must be symmetric. (yes, Gibbs can do asymmetric)

9E2. Gibbs sampling is more efficient than the Metropolis algorithm. How does it achieve this extra
efficiency? Are there any limitations to the Gibbs sampling strategy?


Gibbs is basically gradient descent

(McElreath's words - re-word)

By “efficient,”Imeanthatyoucangetagood
estimateoftheposteriorfromGibbssamplingwithmanyfewersamplesthanacomparable
Metropolisapproach.Theimprovementarisesfrom adaptiveproposals in whichthedistribu-
tionofproposedparametervaluesadjustsitselfintelligently,dependingupontheparameter
valuesatthemoment.

HowGibbssamplingcomputestheseadaptiveproposalsdependsuponusingparticu-
larcombinationsofpriordistributionsandlikelihoodsknownas conjugatepairs. Conjugate
pairshaveanalyticalsolutionsfortheposteriordistributionofanindividualparameter.And
thesesolutionsarewhatallowGibbssamplingtomakesmartjumpsaroundthejointposte-
riordistributionofallparameters.

ButtherearesomeseverelimitationstoGibbssam-
pling.First,maybeyoudon’twanttouseconjugatepriors.Someconjugatepriorsareac-
tuallypathologicalinshape,onceyoustartbuildingmultilevelmodelsandneedpriorsfor
entirecovariancematrixes.

Second, as models become more complex and containhundredsorthousandsortens
ofthousandsofparameters,bothMetropolisandGibbssamplingbecomeshockinglyineffi-
cient.Thereasonisthattheytendtogetstuckinsmallregionsoftheposteriorforpotentially
a longtime. Thehighnumberofparametersisn’ttheproblemsomuchasthefactthatmod-
elswithmanyparametersnearlyalwayshaveregionsofhighcorrelationintheposterior.
Thismeansthattwoormoreparametersarehighlycorrelatedwithoneanotherinthepos-
teriorsamples.You’veseenthisbeforewith,forexample,thetwolegsexamplein Chapter6.
Whyisthisaproblem?Becausehighcorrelationmeansanarrowridgeofhighprobability
combinations,andbothMetropolisandGibbsmaketoomanydumbproposalsofwhereto
gonext.Sotheygetstuck.

9E3. Which sort of parameters can Hamiltonian Monte Carlo not handle? Can you explain why?

Basically what above cannot handle?

Can't work with discrete parameters!


**Limitations.** As always, there are some limitations. HMC requires continuous parameters. It can’t glide through a discrete parameter. **In practice, this means that certain techniques, like the imputation of discrete missing data, have to be done differently with HMC.**

**Remember its the parameters not the outputs that have to be discrete.**

HMC can certainly sample from such models, often much more efficientlythana
Gibbssamplercould.Butyouhavetochangehowyoucodethem.Therewillbeexamples
in Chapter15 and Chapter16.
ItisalsoimportanttokeepinmindthatHMCisnotmagic.Someposteriordistributions
arejustverydifficulttosamplefrom,foranyalgorithm.We’llseeexamplesinlaterchapters.
Inthesecases,HMCwillencountersomethingcalleda divergenttransition. We’lltalk
a lotaboutthese,whatcausesthem,andhowtofixthem,lateron.

9E4. Explain the difference between the effective number of samples, n_eff as calculated by Stan,
and the actual number of samples.

9E5. Which value should Rhat approach, when a chain is sampling the posterior distributioncor-
rectly?

9E6. SketchagoodtraceplotforaMarkovchain,onethatiseffectivelysamplingfromtheposterior
distribution.Whatisgoodaboutitsshape?ThensketchatraceplotforamalfunctioningMarkov
chain.Whataboutitsshapeindicatesmalfunction?
9E7. Repeattheproblemabove,butnowforatracerankplot.

9M1.

The sigma doesn't really change in the posterior. The idea is that the real sigma parameter is in both uniform and exponential.


9M2.

The new prior does matter for beta. It doesn't allow negative values like the previous priors.

Appendix: Environment and system parameters

In [3]:
%watermark -n -u -v -iv -w

Last updated: Wed May 26 2021

Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.20.0

scipy     : 1.6.0
pandas    : 1.2.1
arviz     : 0.11.1
json      : 2.0.9
matplotlib: 3.3.4
pymc3     : 3.11.0
seaborn   : 0.11.1
numpy     : 1.20.1

Watermark: 2.1.0



<IPython.core.display.Javascript object>